# 1 Preparations



## 1.1 Install possibly missing Libraries

In [1]:
!conda install -c conda-forge folium --yes 

Solving environment: done

# All requested packages already installed.



In [2]:
!conda install -c conda-forge geocoder --yes 


Solving environment: done

# All requested packages already installed.



## 1.2 Import libraries

In [3]:
import geocoder # import geocoder
import folium
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn import preprocessing


## 1.3 Define Foursquare credentials  and version

In [4]:
CLIENT_ID = 'TBYPT4P0XQXFSGLKCPKIL1YC23HOSIQXKX2TI3RXCSM0AIR0' # your Foursquare ID
CLIENT_SECRET = '4JB25ZUTYPGXESNCGJAOFTM5WAWHBS0NIK3XDDBXZQ5RC2NQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: TBYPT4P0XQXFSGLKCPKIL1YC23HOSIQXKX2TI3RXCSM0AIR0
CLIENT_SECRET:4JB25ZUTYPGXESNCGJAOFTM5WAWHBS0NIK3XDDBXZQ5RC2NQ


In [5]:
CLIENT_ID = 'TYHKWOZVUQREHMY535EMAP1D4E4C4KOPMSIIBXQEQVAITTO1' # your Foursquare ID
CLIENT_SECRET = '5TMUBLRQCYXDOAIGMFSIDDD1AU3SLF5BARBVJ0S51KXWIT4H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: TYHKWOZVUQREHMY535EMAP1D4E4C4KOPMSIIBXQEQVAITTO1
CLIENT_SECRET:5TMUBLRQCYXDOAIGMFSIDDD1AU3SLF5BARBVJ0S51KXWIT4H


# 1.4 Define function for getting the nearby values for all the neighborhoods and configure it


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=1750,LIMIT = 250):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

LIMIT = 450


# 2. Create Dataframe with Neighbourhooddata

## 2.1 Define Wikipedia page to scrape data from, get soup and find table (it is the second one)

In [7]:
website_url = requests.get('https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Ortsteile_Berlins').text
soup = BeautifulSoup(website_url,'lxml')
My_table = soup.find_all('table',{'class':'wikitable sortable zebra'})[1]

## 2.2 Extract the data from the table and find the coordinates for each neihbourhood applying geocoder

In [8]:
cods  = []
neis  = []
bors  = []
area  = []
inha  = []
inar  = []
longs = []
lats  = []

gc = Nominatim(user_agent="fintu-blog-geocoding-python")
b=1

for row in My_table.find_all('tr'):
    column = row.find_all('td')
    if not column:
        continue
    cod, nei, bor, are, inh,ina = [cell.text.strip() for cell in column[:6]]
    if bor == 'Not assigned':
        continue
    if nei == 'Not assigned':
        nei = bor
    else:
        cods.append(cod)
        neis.append(nei)
        bors.append(bor)
        area.append(are)
        inha.append(inh)
        inar.append(ina)
        cords = gc.geocode(nei + ", Berlin")
        longs.append(cords.longitude)
        lats.append(cords.latitude)
    print(b)
    b=b+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96


## 2.3 Construct dataframe from lists and check shape

In [9]:
berlin_data = pd.DataFrame(list(zip(cods, neis,bors, area,inha,inar,longs,lats)), 
               columns=['Number','Neighbourhood','Borough', 'Area', 'Inhabitants','Inhabitants per sqkm','Longitude','Latitude'])
berlin_data.shape

berlin_data

,Number,Neighbourhood,Borough,Area,Inhabitants,Inhabitants per sqkm,Longitude,Latitude
0,0101,Mitte,Mitte,"10,7",99.998,9346,13.402376,52.517690
1,0102,Moabit,Mitte,"7,72",78.491,10.167,13.342542,52.530102
2,0103,Hansaviertel,Mitte,"0,53",5.831,11.002,13.341872,52.519123
3,0104,Tiergarten,Mitte,"5,17",14.529,2810,13.357260,52.509778
4,0105,Wedding,Mitte,"9,23",86.468,9368,13.341970,52.550123
5,0106,Gesundbrunnen,Mitte,"6,13",94.293,15.382,13.384846,52.550920
6,0201,Friedrichshain,Friedrichshain-Kreuzberg,"9,78",131.953,13.492,13.450290,52.512215
7,0202,Kreuzberg,Friedrichshain-Kreuzberg,"10,4",154.010,14.809,13.411914,52.497644
8,0301,Prenzlauer Berg,Pankow,"11,0",163.481,14.862,13.428565,52.539847
9,0302,Weißensee,Pankow,"7,93",53.614,6761,13.463002,52.554619


# 3 Exploring the data

## 3.1 Map of Berlin with markers for the neighbourhoods
    


In [10]:
map_Berlin = folium.Map(location=[lats[0], longs[0]], zoom_start=11) #Starting with the coords of Berlin Central

# add markers to map
for lat, lng, label in zip(berlin_data['Latitude'], berlin_data['Longitude'], berlin_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Berlin)  
    
map_Berlin


## 3.2 Getting the venues for each neighbourhood applying the function from 1.4

In [11]:
berlin_venues = getNearbyVenues(names=berlin_data['Neighbourhood'],
                                   latitudes=berlin_data['Latitude'],
                                   longitudes=berlin_data['Longitude']
                                  )


Mitte
Moabit
Hansaviertel
Tiergarten
Wedding
Gesundbrunnen
Friedrichshain
Kreuzberg
Prenzlauer Berg
Weißensee
Blankenburg
Heinersdorf
Karow
Stadtrandsiedlung Malchow
Pankow
Blankenfelde
Buch
Französisch Buchholz
Niederschönhausen
Rosenthal
Wilhelmsruh
Charlottenburg
Wilmersdorf
Schmargendorf
Grunewald
Westend
Charlottenburg-Nord
Halensee
Spandau
Haselhorst
Siemensstadt
Staaken
Gatow
Kladow
Hakenfelde
Falkenhagener Feld
Wilhelmstadt
Steglitz
Lichterfelde
Lankwitz
Zehlendorf
Dahlem
Nikolassee
Wannsee
Schöneberg
Friedenau
Tempelhof
Mariendorf
Marienfelde
Lichtenrade
Neukölln
Britz
Buckow
Rudow
Gropiusstadt
Alt-Treptow
Plänterwald
Baumschulenweg
Johannisthal
Niederschöneweide
Altglienicke
Adlershof
Bohnsdorf
Oberschöneweide
Köpenick
Friedrichshagen
Rahnsdorf
Grünau
Müggelheim
Schmöckwitz
Marzahn
Biesdorf
Kaulsdorf
Mahlsdorf
Hellersdorf
Friedrichsfelde
Karlshorst
Lichtenberg
Falkenberg
Malchow
Wartenberg
Neu-Hohenschönhausen
Alt-Hohenschönhausen
Fennpfuhl
Rummelsburg
Reinickendorf
Tegel
Kon

## 3.3 Analysing shape and head of the results

In [12]:
print(berlin_venues.shape)
berlin_venues.head()


(4724, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mitte,52.51769,13.402376,Lustgarten,52.518469,13.399454,Garden
1,Mitte,52.51769,13.402376,Radisson Blu,52.519580,13.402730,Hotel
2,Mitte,52.51769,13.402376,Kuppelumgang Berliner Dom,52.518966,13.400981,Scenic Lookout
3,Mitte,52.51769,13.402376,"Bronzestatue ""Heiliger St. Georg im Kampf mit ...",52.516290,13.405558,Outdoor Sculpture
4,Mitte,52.51769,13.402376,Deutsches Historisches Museum,52.517788,13.396948,History Museum


## 3.4 Filtering the dataframe to get only venues related to climbing

In [13]:
berlin_venues[berlin_venues["Venue Category"].str.contains("Climb")]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
506,Gesundbrunnen,52.550920,13.384846,Magic Mountain Kletterhalle,52.548613,13.381902,Climbing Gym
661,Friedrichshain,52.512215,13.450290,Der Kegel,52.507287,13.454390,Climbing Gym
1625,Charlottenburg-Nord,52.540525,13.296266,Waldhochseilgarten Jungfernheide,52.543001,13.290644,Rock Climbing Spot
2478,Schöneberg,52.482157,13.355190,Bright Site,52.481732,13.365768,Climbing Gym
2760,Mariendorf,52.440080,13.390028,Südbloc Boulderhalle,52.439969,13.379566,Climbing Gym
2845,Marienfelde,52.412577,13.366592,Area 85 - South Rock Kletterhalle,52.427086,13.369916,Climbing Gym
3170,Alt-Treptow,52.492563,13.459874,Bouldergarten,52.479470,13.451368,Climbing Gym
3215,Plänterwald,52.479544,13.478808,Ostbloc Boulderhalle,52.491496,13.487964,Climbing Gym
3796,Hellersdorf,52.536854,13.604774,BergWerk,52.537724,13.603606,Rock Climbing Spot
4255,Rummelsburg,52.501370,13.483514,Ostbloc Boulderhalle,52.491496,13.487964,Climbing Gym


## 3.5 Create a list with all neighborhoods with climbing halls

In [14]:
neigh_with_climb= berlin_venues[berlin_venues["Venue Category"].str.contains("Climb")]["Neighborhood"].tolist()
neigh_with_climb

['Gesundbrunnen',
 'Friedrichshain',
 'Charlottenburg-Nord',
 'Schöneberg',
 'Mariendorf',
 'Marienfelde',
 'Alt-Treptow',
 'Plänterwald',
 'Hellersdorf',
 'Rummelsburg']

## 3.6 Creating a map showing the neighborhoods with climbing halls


In [15]:
map_Berlin = folium.Map(location=[lats[0], longs[0]], zoom_start=11) #Starting with the coords of Berlin Central

# add markers to map
for lat, lng, label in zip(berlin_data['Latitude'], berlin_data['Longitude'], berlin_data['Neighbourhood']):
    this_color ="blue"
    if label in neigh_with_climb:
        this_color ="red"
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=this_color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Berlin)  
    
map_Berlin


# 4 Finding good locations

# 4.1 Finding the biggest neighbourhoods

In [16]:
berlin_venues.groupby('Neighborhood').count().sort_values("Venue", ascending= False)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Gesundbrunnen,100,100,100,100,100,100
Tiergarten,100,100,100,100,100,100
Halensee,100,100,100,100,100,100
Hansaviertel,100,100,100,100,100,100
Rummelsburg,100,100,100,100,100,100
Prenzlauer Berg,100,100,100,100,100,100
Friedrichshain,100,100,100,100,100,100
Friedenau,100,100,100,100,100,100
Steglitz,100,100,100,100,100,100


## 4.2 Finding the venues close to climbing halls



### 4.2.1 Splitting the dataframe based on wheather there are climbing halls located within

In [17]:
berlin_venues_climb = berlin_venues[berlin_venues.Neighborhood.isin(neigh_with_climb)]
berlin_venues_climb

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
500,Gesundbrunnen,52.55092,13.384846,La Femme,52.550107,13.385304,Breakfast Spot
501,Gesundbrunnen,52.55092,13.384846,F-Bar,52.551908,13.386493,Dive Bar
502,Gesundbrunnen,52.55092,13.384846,MOXY Berlin Humboldthain Park,52.549568,13.384071,Hotel
503,Gesundbrunnen,52.55092,13.384846,Il Milanese del Tacco,52.551772,13.384107,Gourmet Shop
504,Gesundbrunnen,52.55092,13.384846,REWE Center,52.549751,13.387445,Supermarket
505,Gesundbrunnen,52.55092,13.384846,Berliner Unterwelten e.V.,52.547745,13.388864,History Museum
506,Gesundbrunnen,52.55092,13.384846,Magic Mountain Kletterhalle,52.548613,13.381902,Climbing Gym
507,Gesundbrunnen,52.55092,13.384846,Rosengarten,52.547565,13.387207,Garden
508,Gesundbrunnen,52.55092,13.384846,Berliner Unterwelten Tour 1,52.547657,13.388873,Trail
509,Gesundbrunnen,52.55092,13.384846,Curry Baude,52.549420,13.386999,Currywurst Joint


### 4.2.2 Find common venues in neighbourhoods with climbing halls

In [18]:
# one hot encoding
berlin_climb_onehot = pd.get_dummies(berlin_venues_climb[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
berlin_climb_onehot['Neighbourhood'] = berlin_venues_climb['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [berlin_climb_onehot.columns[-1]] + list(berlin_climb_onehot.columns[:-1])
berlin_climb_onehot = berlin_climb_onehot[fixed_columns]

berlin_climb_total =  berlin_climb_onehot.mean().reset_index()
berlin_climb_total.sort_values(0, ascending= False)

,index,0
36,Café,0.059355
192,Supermarket,0.046452
14,Bar,0.037419
45,Coffee Shop,0.029677
141,Park,0.024516
136,Nightclub,0.023226
12,Bakery,0.023226
150,Pizza Place,0.020645
112,Italian Restaurant,0.020645
106,Ice Cream Shop,0.018065


### 4.2.3 Find common venues in average neighbourhoods

In [19]:
# one hot encoding
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
berlin_onehot['Neighbourhood'] = berlin_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]

berlin_total =  berlin_onehot.mean().reset_index()

berlin_total.sort_values(0, ascending= False)


,index,0
310,Supermarket,0.087003
59,Café,0.047206
176,Italian Restaurant,0.036198
163,Hotel,0.025402
231,Park,0.025402
140,German Restaurant,0.023920
24,Bakery,0.023709
105,Drugstore,0.022650
54,Bus Stop,0.020322
166,Ice Cream Shop,0.020110


### 4.2.4. Outer join to get a dataframe with frequencies for both subsets

In [20]:
berlin_difference = pd.merge(berlin_total, berlin_climb_total, on='index', how='outer')
berlin_difference = berlin_difference.fillna(0)
berlin_difference = berlin_difference.rename(columns={"Total": "0_x", "Climb": "0_y"})
berlin_difference[1] = berlin_difference["0_y"] - berlin_difference["0_x"]

### 4.2.5 Fiding the special features of neighbourhoods with climbing halls


In [21]:
berlin_difference.sort_values(1, ascending= False)


,index,0_x,0_y,1
26,Bar,0.017570,0.037419,0.019849
220,Nightclub,0.005292,0.023226,0.017934
59,Café,0.047206,0.059355,0.012149
75,Coffee Shop,0.019052,0.029677,0.010626
72,Climbing Gym,0.001693,0.010323,0.008629
269,Rock Club,0.001693,0.010323,0.008629
335,Vegetarian / Vegan Restaurant,0.005927,0.012903,0.006976
246,Pizza Place,0.013760,0.020645,0.006886
4,Airport Service,0.001270,0.007742,0.006472
344,Wine Bar,0.004022,0.010323,0.006301


### 4.2.6 Calculate score for all neighbourhoods

In [24]:
berlin_grouped = berlin_onehot.groupby('Neighbourhood').mean().reset_index()
berlin_grouped_score["Pos Score"] = berlin_grouped["Bar"] + berlin_grouped["Nightclub"] + berlin_grouped["Café"] + berlin_grouped["Coffee Shop"] +berlin_grouped["Climbing Gym"] + berlin_grouped["Rock Club"] +berlin_grouped["Vegetarian / Vegan Restaurant"] + berlin_grouped["Pizza Place"]+ berlin_grouped["Airport Service"] + berlin_grouped["Wine Bar"] 
berlin_grouped_score["Neg Score"] = berlin_grouped["Gas Station"] + berlin_grouped["Plaza"] + berlin_grouped["Trattoria/Osteria"] + berlin_grouped["Greek Restaurant"] +berlin_grouped["Tram Station"] + berlin_grouped["German Restaurant"] +berlin_grouped["Drugstore"] + berlin_grouped["Hotel"]+ berlin_grouped["Italian Restaurant"] + berlin_grouped["Supermarket"] 
berlin_grouped_score["Tot Score"] = berlin_grouped_score["Pos Score"] - berlin_grouped_score["Neg Score"]
berlin_grouped_score[["Neighbourhood","Pos Score","Neg Score","Tot Score"]].sort_values("Tot Score", ascending= False)

,Neighbourhood,Pos Score,Neg Score,Tot Score
23,Friedrichshain,0.330000,0.050000,0.280000
26,Gesundbrunnen,0.320000,0.040000,0.280000
61,Neukölln,0.350000,0.070000,0.280000
2,Alt-Treptow,0.280000,0.040000,0.240000
73,Rummelsburg,0.300000,0.080000,0.220000
44,Kreuzberg,0.280000,0.060000,0.220000
88,Wedding,0.280000,0.110000,0.170000
7,Blankenfelde,0.142857,0.000000,0.142857
76,Schöneberg,0.230000,0.100000,0.130000
68,Prenzlauer Berg,0.190000,0.070000,0.120000


## 4.3 Classifying neighbourhoods with logistic regression

### 4.3.1 Grouping the venues by neighbourhoods and adding a column stating if there is a climbing hall

In [25]:
berlin_grouped = berlin_onehot.groupby('Neighbourhood').mean().reset_index()
berlin_grouped["AA_y"] = 0
berlin_grouped["AA_y"].loc[berlin_grouped.Neighbourhood.isin(neigh_with_climb)] = 1
berlin_grouped

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Neighbourhood,ATM,Adult Boutique,African Restaurant,Airport Lounge,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Aquarium,...,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit,AA_y
0,Adlershof,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
1,Alt-Hohenschönhausen,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
2,Alt-Treptow,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.010000,0.00,0.00,0.00,0.010000,0.000000,0.000000,1
3,Altglienicke,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
4,Baumschulenweg,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
5,Biesdorf,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.058824,0
6,Blankenburg,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
7,Blankenfelde,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
8,Bohnsdorf,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0
9,Borsigwalde,0.000000,0.012048,0.00,0.000000,0.000000,0.000000,0.0,0.0000,0.00,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0


### 4.3.2 Splitting the dataframe in X_train and Y_train and normalize X_train data

In [26]:
X_train= berlin_grouped.drop(columns=['AA_y', 'Neighbourhood'])
Y_train=berlin_grouped["AA_y"]
X_train= preprocessing.StandardScaler().fit(X_train).transform(X_train)

### 4.3.3 Perform logistic regression

In [27]:
LR = LogisticRegression(C=0.001, solver='liblinear').fit(X_train,Y_train)

### 4.3.4 Classify neighbourhoods with logistic regression and show suitable neighbourhoods

In [28]:
yhat = LR.predict(X_train)
yhat_prob = LR.predict_proba(X_train)
berlin_results= berlin_grouped[["Neighbourhood","AA_y"]]
berlin_results["yhat"] = yhat
berlin_results["yhat_prob"] = yhat_prob[:,1]
berlin_results["diff"] = berlin_results["yhat"] - berlin_results["AA_y"]
berlin_results[berlin_results.yhat == 1]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

,Neighbourhood,AA_y,yhat,yhat_prob,diff
2,Alt-Treptow,1,1,0.691761,0
14,Charlottenburg-Nord,1,1,0.649241,0
18,Fennpfuhl,0,1,0.506025,1
23,Friedrichshain,1,1,0.712928,0
26,Gesundbrunnen,1,1,0.665096,0
36,Hellersdorf,1,1,0.571083,0
44,Kreuzberg,0,1,0.511346,1
53,Mariendorf,1,1,0.601803,0
54,Marienfelde,1,1,0.544411,0
56,Mitte,0,1,0.513817,1


### 4.3.5 Calculate accuracy metrics

In [29]:
j4   = jaccard_similarity_score(Y_train, yhat)
f4   = f1_score(Y_train, yhat, average='weighted') 
ll4  = log_loss(Y_train, yhat_prob)


In [30]:
j4

0.9375

In [31]:
f4

0.9435820203892492

In [32]:
ll4

0.6273415660884804

### 4.3.6 Determine neighbourhoods which are classified as suitabel for climbing halls although there is no hall yet

In [33]:
new = yhat - Y_train
new_neighbourhoods = berlin_grouped["Neighbourhood"].loc[new == 1]
new_neighbourhoods

18    Fennpfuhl
44    Kreuzberg
56        Mitte
61     Neukölln
83    Tempelhof
88      Wedding
Name: Neighbourhood, dtype: object